In [ ]:
import requests
import pandas as pd
import os
os.makedirs("rechercheZenodo", exist_ok=True)

ACCESS_TOKEN = 'indiquer le token ici'
url = 'https://zenodo.org/api/records'

# Paramètres de la requête
params = {
    'q': 'Inria, Institut national de recherche en informatique et en automatique',
    'access_token': ACCESS_TOKEN,
    'sort': 'bestmatch',
    'size': 100,  # Limite à 100 résultats par page
    'publication_date': '*'
}

# Fonction pour récupérer les publications
def get_publications():
    publications = []
    page = 1
    while True:
        params['page'] = page
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            hits = data['hits']['hits']
            if not hits:
                break
            # Extraire les informations demandées
            for record in hits:
                creators = record['metadata'].get('creators', [])
                # Concaténation des informations des créateurs (nom, affiliation, orcid)
                creators_info = ', '.join([f"{creator.get('name', 'N/A')} ({creator.get('affiliation', 'N/A')}, {creator.get('orcid', 'N/A')})" for creator in creators])

                # Institution: Concaténer toutes les affiliations des auteurs, en évitant les None
                institutions = ', '.join([creator.get('affiliation') for creator in creators if creator.get('affiliation')])  # Inclut seulement les affiliations non None ou non vides

                # Récupérer le type de ressource si disponible
                publication_type = record['metadata'].get('resource_type', {}).get('type', 'N/A')

                # Récupération des sujets (si disponibles)
                subjects = record['metadata'].get('subjects', [])
                if subjects:
                    subjects_info = ', '.join([f"{subject.get('term', 'N/A')} (ID: {subject.get('identifier', 'N/A')}, Scheme: {subject.get('scheme', 'N/A')})" for subject in subjects])
                else:
                    subjects_info = 'N/A'

                # Récupérer les keywords (mots-clés) si disponibles
                keywords = record['metadata'].get('keywords', [])
                if keywords:
                    keywords_info = ', '.join(keywords)
                else:
                    keywords_info = 'N/A'

                # Créer l'URL de Zenodo en utilisant l'identifiant de l'enregistrement
                record_id = record['id']
                url_zenodo = f"https://zenodo.org/records/{record_id}"

                publications.append({
                    'title': record['metadata'].get('title', 'N/A'),
                    'publication_date': record['metadata'].get('publication_date', 'N/A'),
                    'doi': record['metadata'].get('doi', 'N/A'),
                    'creators': creators_info,
                    'publication_type': publication_type,
                    'access_right': record['metadata'].get('access_right', 'N/A'),
                    'institution': institutions if institutions else 'N/A',  # Toutes les affiliations
                    'subjects': subjects_info,  # Ajout des sujets formatés correctement
                    'keywords': keywords_info,  # Ajout des mots-clés (keywords)
                    'url_zenodo': url_zenodo  # URL Zenodo pour chaque publication
                })
            page += 1
        else:
            print(f"Erreur lors de la récupération des données: {response.status_code}")
            break
    return publications

# Récupérer les publications et les mettre dans un DataFrame
publications = get_publications()
df_publications = pd.DataFrame(publications)

df_publications


Erreur lors de la récupération des données: 400


,title,publication_date,doi,creators,publication_type,access_right,institution,subjects,keywords,url_zenodo
0,DEFI-COLaF/LADaS: 2024-02-20 - First release,2024-02-20,10.5281/zenodo.10682624,"Clérice, Thibault (Institut national de recher...",dataset,open,Institut national de recherche en informatique...,N/A,N/A,https://zenodo.org/records/10682624
1,Interpreting Vegetation-Climate Interaction wi...,2024-04-26,10.5281/zenodo.11072948,"Ricardo, Barros Lourenço (McMaster University,...",presentation,open,McMaster University,N/A,N/A,https://zenodo.org/records/11072948
2,Data-Efficient French Language Modeling with C...,2023-08-02,10.18653/v1/2023.findings-acl.320,"Antoun, Wissam (Institut national de recherche...",publication,open,Institut national de recherche en informatique...,N/A,N/A,https://zenodo.org/records/10024064
3,Towards a Robust Detection of Language Model G...,2023-07-12,10.5281/zenodo.10024578,"Seddah, Djamé (Institut national de recherche ...",publication,open,Institut national de recherche en informatique...,N/A,N/A,https://zenodo.org/records/10024578
4,Enriching the NArabizi Treebank: A Multifacete...,2023-06-10,10.18653/v1/2023.law-1.26,"Riabi, Arij (Institut national de recherche en...",publication,open,Institut national de recherche en informatique...,N/A,N/A,https://zenodo.org/records/10024092
...,...,...,...,...,...,...,...,...,...,...
9995,Caractérisation de la pêche récifale par l'ana...,2025-02-06,10.5281/zenodo.14824404,"Behivoke, Faustinato (Institut Halieutique et ...",publication,open,Institut Halieutique et des Sciences Marines (...,N/A,"GPS, forêt aléatoire, apprentissage, effort de...",https://zenodo.org/records/14824404
9996,"FIG. 7 in Le genre Eustra Schmidt-Goebel, 1846...",2001-12-31,10.5281/zenodo.5391043,"Deuve, Thierry (Muséum national d'Histoire nat...",image,open,"Muséum national d'Histoire naturelle, Laborato...",N/A,"Biodiversity, Taxonomy, Animalia, Arthropoda, ...",https://zenodo.org/records/5391043
9997,LA PNEUMOPATHIE INTERSTITIELLE DIFFUSE AU COUR...,2024-06-15,10.21474/IJAR01/18809,"Amal El-Ouakhoumi , Jaouad Yousfi , Mouna Zahl...",dataset,open,"Service de Medecine Interne, Centre Hospitalie...",N/A,N/A,https://zenodo.org/records/11667071
9998,MARTIALISATION DU CORPS ET DES DIFFERENTES PA...,2023-12-16,10.5281/zenodo.10394925,"TCHAOU, Paterne Djidéwou (Université d'Abomey-...",publication,open,"Université d'Abomey-Calavi, Bénin",N/A,N/A,https://zenodo.org/records/10394925


In [ ]:
df_publications.to_csv("rechercheZenodo/zenodo_inria_publications.csv", index=False)